---
# Demo Use Case - Ingest Live Video Streams (Part 1)

![overview](adas_overview.png)

A data scientist is part of a team developing an advanced driver-assistance system (ADAS).
They continuously collect video, lidar, and other sensor data from their fleet of test vehicles.
The data scientist wants to test a new object detection model on video collected today.

The purpose of this Jupyter notebook is to simulate the ingestion of real-time video into Streaming Data Platform (SDP).

To avoid the need for a set of live cameras for this demo, we play back video from a series of JPEG files on disk
and write each video frame to SDP.
These videos are part of the [KITTI Vision Benchmark Suite](http://www.cvlibs.net/datasets/kitti/raw_data.php).

- We simulate up to 12 simultaneous cameras from different vehicles.
- Each video frame is 1242 x 374 pixels, RGB color, JPEG format.
- JPEG image sizes average 50 KB.
- Each camera records at 10 frames per second.
- The total ingest rate is 2.1 MB/sec.

---

### Prerequisites

1. [install_dependencies.ipynb](install_dependencies.ipynb).
2. [adas_download.ipynb](adas_download.ipynb).
3. [adas_transcode_from_png.ipynb](adas_transcode_from_png.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, opencv_image_to_mpl, ImageFileSequenceLoader
import os

### Define Pravega stream parameters

In [ ]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
stream = 'adas-raw-video'
#stream = 'object-detector-input-video'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [ ]:
#pravega_client.CreateScope(pravega.pb.CreateScopeRequest(scope=scope))

### Create Pravega stream

In [ ]:
output_stream = OutputStream(pravega_client, scope, stream)
#output_stream.delete_stream()
output_stream.create_stream(min_num_segments=1)
output_stream.truncate_stream()

In [ ]:
# Truncate object detector output stream
detected_stream = OutputStream(pravega_client, scope, 'object-detector-output-video')
#detected_stream.delete_stream()
detected_stream.create_stream(min_num_segments=1)
detected_stream.truncate_stream()

### Ingest JPEG images from files (multiple cameras)

In [ ]:
prefix = '../../data/'
camera_filespecs = [
    prefix + 'kitti/2011_09_26/2011_09_26_drive_0005_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0009_sync/image_02/data/*.jpg',
#     '../../data/adas/traffic-flow-in-the-highway-2103099/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0011_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0051_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0009_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0011_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0014_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0051_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0059_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_26/2011_09_26_drive_0084_sync/image_02/data/*.jpg',
#     prefix + 'kitti/2011_09_29/2011_09_29_drive_0071_sync/image_02/data/*.jpg',
]
fps = 10  # 10 for kitti
step = 1

In [ ]:
#camera_filespecs = camera_filespecs * 10

In [ ]:
loader = ImageFileSequenceLoader(scope, stream, camera_filespecs, fps, step=step)
events_to_write = loader.event_generator()
output_stream.write_events(events_to_write)